In [1]:
from updated_qsp_dataset import UpdatedWebQSPDataset
from raw_qsp_dataset import RawWebQSPDataset

/home/zaristei/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
old_dataset = RawWebQSPDataset(root='old_dataset', force_reload=True, limit=2, with_process=True, with_pcst=True)
new_dataset = UpdatedWebQSPDataset(root='updated_dataset',force_reload=True, limit=2)

Processing...


inherit model weights from sentence-transformers/all-roberta-large-v1
Encoding questions...
Encoding graphs...


100%|██████████| 2/2 [00:08<00:00,  4.30s/it]
Done!
Processing...


inherit model weights from sentence-transformers/all-roberta-large-v1
Encoding graph...
Encoding questions...
Retrieving subgraphs...


100%|██████████| 2/2 [00:00<00:00,  3.56it/s]
Done!


In [3]:
new_dataset[0]

Data(x=[15, 1024], edge_index=[2, 20], edge_attr=[20, 1024], question='Question: what is the name of justin bieber brother
Answer: ', label='jaxon bieber', desc='node_id,node_attr
249,m.0129jzth
434,jeremy bieber
880,m.0gxnp5d
892,m.0gxnnwc
1178,justin bieber
1239,yves bole
1563,english language
1569,jazmyn bieber
1742,pattie mallette
1858,m.0gxnp5m
1888,this is justin bieber
2050,jaxon bieber
2205,m.0gxnnwp
2436,m.0wfn4pm
2620,m.0gxnp5x

src,edge_attr,dst
1239,people.person.languages,1563
1888,film.film.language,1563
1239,influence.influence_node.influenced_by,1178
1742,people.person.children,1178
2050,people.person.parents,434
2436,people.sibling_relationship.sibling,1742
2050,people.person.sibling_s,2205
1178,base.popstra.celebrity.hangout,2620
892,people.sibling_relationship.sibling,1569
249,people.sibling_relationship.sibling,1239
1742,people.person.sibling_s,2436
2205,people.sibling_relationship.sibling,2050
1239,people.person.sibling_s,249
1178,people.person.sibling_s,892
1178,b

In [4]:
old_dataset[0]

Data(x=[12, 1024], edge_index=[2, 13], edge_attr=[13, 1024], question='Question: what is the name of justin bieber brother
Answer: ', label='jaxon bieber', desc='node_id,node_attr
15,justin bieber
151,pattie mallette
286,english language
294,jaxon bieber
346,yves bole
356,jeremy bieber
452,jazmyn bieber
545,m.0wfn4pm
551,m.0gxnnwp
933,m.0gxnnwc
1032,this is justin bieber
1359,m.0129jzth

src,edge_attr,dst
346,people.person.languages,286
1032,film.film.language,286
346,influence.influence_node.influenced_by,15
151,people.person.children,15
294,people.person.parents,356
294,people.person.sibling_s,551
452,people.person.sibling_s,933
545,people.sibling_relationship.sibling,151
933,people.sibling_relationship.sibling,452
1359,people.sibling_relationship.sibling,346
551,people.sibling_relationship.sibling,294
933,people.sibling_relationship.sibling,15
551,people.sibling_relationship.sibling,15
', num_nodes=12)

In [ ]:
new_dataset.raw_graphs[0]

In [ ]:
old_dataset.raw_graphs[0]

In [ ]:
import networkx as nx
from torch_geometric.data import Data
import torch

In [ ]:
def results_are_close_enough(ground_truth: Data, new_method: Data, thresh=.8):
    def _sorted_tensors_are_close(tensor1, tensor2):
        return torch.all(torch.isclose(tensor1.sort(dim=0)[0], tensor2.sort(dim=0)[0]).float().mean(axis=1) > thresh)
    def _graphs_are_same(tensor1, tensor2):
        return nx.weisfeiler_lehman_graph_hash(nx.Graph(tensor1.T)) == nx.weisfeiler_lehman_graph_hash(nx.Graph(tensor2.T))
    return _sorted_tensors_are_close(ground_truth.x, new_method.x) \
        and _sorted_tensors_are_close(ground_truth.edge_attr, new_method.edge_attr) \
        and _graphs_are_same(ground_truth.edge_index, new_method.edge_index)

In [ ]:
results_are_close_enough(old_dataset.raw_graphs[0], new_dataset.raw_graphs[0])

In [ ]:
old_dataset.questions[0]

In [ ]:
new_dataset.questions[0]